In [15]:
import numpy as np
import astroplan

from matplotlib import pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
import pytz
from astroplan import Observer, FixedTarget
from astroplan import plots
color_cycle = plots.mplstyles.astropy_mpl_style['axes.color_cycle']

In [16]:
print(astroplan.get_site('MRO'))

(-2228982.313119391, -3749888.25304088, 4639060.0864693) m


In [17]:
# from here on is trying to find the best night to observe "TYC 30-78-1" from MRO:
# the best night being one allowing for the longest continuous observation with the moon 70+ degrees away
mro = Observer.at_site('MRO', timezone=pytz.timezone('US/Pacific'))
print(mro)

<Observer: name='MRO',
    location (lon, lat, el)=(-120.7278 deg, 46.9528 deg, 1198.00000000122 m),
    timezone=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>>


In [36]:
coordinates = SkyCoord('01h54m38.73s', '+00d43m00.57s', frame='icrs')
targets = [FixedTarget(coords,'TYC 30-78-1')]
print(targets)

[<FixedTarget "TYC 30-78-1" at SkyCoord (ICRS): (ra, dec) in deg (15.91055556, 0.71666667)>]


In [19]:
time_range = Time(["2016-06-09 00:00", "2016-10-14 00:00"])
from astroplan import (AltitudeConstraint, AirmassConstraint, AtNightConstraint, MoonSeparationConstraint)
constraints = [AltitudeConstraint(10*u.deg, 80*u.deg), AirmassConstraint(3), AtNightConstraint.twilight_civil(), MoonSeparationConstraint(70*u.deg)]
from astroplan import is_observable, is_always_observable
always_observable = is_always_observable(constraints, mro, targets, time_range=time_range)

In [29]:
from astroplan import observability_table
from astroplan import time_grid_from_range
time_grid = time_grid_from_range(time_range)
print(time_grid)
#print(is_observable(constraints, mro, targets, times = time_grid))

for n in range(len(time_grid)-1):
    if is_observable(constraints, mro, targets, time_range=[time_grid[n],time_grid[n+1]]):
        print(time_grid[n].iso,time_grid[n+1].iso, 'observable')
        #table = observability_table(constraints, mro, targets, time_range=[time_grid[n],time_grid[n+1]])
        #print(table)
        

[ 2457548.5         2457548.52083333  2457548.54166667 ...,
  2457675.43750095  2457675.45833428  2457675.47916761]
2016-06-14 11:00:00.004 2016-06-14 11:30:00.004 observable
2016-06-14 11:30:00.004 2016-06-14 12:00:00.004 observable
2016-06-15 11:00:00.004 2016-06-15 11:30:00.004 observable
2016-06-15 11:30:00.004 2016-06-15 12:00:00.004 observable
2016-06-16 11:00:00.005 2016-06-16 11:30:00.005 observable
2016-06-16 11:30:00.005 2016-06-16 12:00:00.005 observable
2016-06-17 11:00:00.005 2016-06-17 11:30:00.005 observable
2016-06-17 11:30:00.005 2016-06-17 12:00:00.005 observable
2016-06-18 11:00:00.006 2016-06-18 11:30:00.006 observable
2016-06-18 11:30:00.006 2016-06-18 12:00:00.006 observable
2016-06-19 11:00:00.007 2016-06-19 11:30:00.007 observable
2016-06-19 11:30:00.007 2016-06-19 12:00:00.007 observable
2016-06-20 11:00:00.007 2016-06-20 11:30:00.007 observable
2016-06-20 11:30:00.007 2016-06-20 12:00:00.007 observable
2016-06-21 11:00:00.008 2016-06-21 11:30:00.008 observable

In [35]:
longest_time = Time(['2016-10-09 04:00:00','2016-10-09 12:00:00'])
is_always_observable(constraints, mro, targets, time_range=longest_time)

array([ True], dtype=bool)

In [40]:
local_time = longest_time - 7*u.hour #i happen to know MRO is UTC-7hr

<Time object: scale='utc' format='iso' value=['2016-10-08 21:00:00.000' '2016-10-09 05:00:00.000']>

In [54]:
meridian_time = mro.target_meridian_transit_time(longest_time[0],targets)
local_meridian = meridian_time-7*u.hour
mro.target_hour_angle(longest_time[0],targets)
print(mro.moon_altaz(meridian_time))

<SkyCoord (AltAz: obstime=[ 2457670.82914489], location=(-2228982.313119391, -3749888.25304088, 4639060.0864693) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0, obswl=1.0 micron): (az, alt) in deg
    (255.49426349, -12.09660994)>
